In [4]:
import os
import json

In [5]:
def figure_out_csharp(json_lists):
#     print(len(json_lists))
    all_tokens = set()
    children = {}
    parents = {}
    max_vars = 0
    max_consts = 0
    max_chars = 0
    max_strings = 0
    
    def tally_tokens(prog, parent=None):
#         print("doing stuff", type(prog))
#         print(prog)
        all_vars = set()
        all_consts = set()
        all_chars = set()
        all_strings = set()
        
        if type(prog) == type("") or type(prog) == type(1):
            # Add to parents
            if prog in parents:
                parents[prog].add(parent)
            else:
                parents[prog] = set([parent])
                
            return all_vars, all_consts, all_chars, all_strings
        
        
#         print(type(prog))
        for key in prog.keys():
             
            root_val = key
            if not type(root_val) == type(""):
                print("AAAAAAA")
#             print("root", root_val)
            # Add to all tokens
            all_tokens.add(root_val)
            # Add to parents
            if root_val in parents:
                parents[root_val].add(parent)
            else:
                parents[root_val] = set([parent])
            # Add to children
            if parent in children:
                children[parent].add(root_val)
            else:
                children[parent] = set([root_val])
                
            if root_val == "IdentifierName":
                all_vars.add(prog[key])
            elif root_val == "NumericLiteralExpression":
                all_consts.add(prog[key])
            elif root_val == "CharacterLiteralExpression":
                all_chars.add(prog[key])
            elif root_val == "StringLiteralExpression":
                print("got a string")
                all_strings.add(prog[key])
            else:
                child_keys = []
                
                if type(prog[key]) == type([]):
                    for child in prog[key]:
                        for child_key in child.keys():
                            child_keys.append((child_key, child))
                elif type(prog[key]) == type(""):
                    child_keys.append((prog[key], prog[key]))
                else:
                    print("something weird is going on", prog)
                
                # Add to children
                for c in child_keys:
                    child_key, child = c
                    if root_val in children:
                        children[root_val].add(child_key)
                    else:
                        children[root_val] = set([child_key])
                            
                    child_vars, child_consts, child_chars, child_strings = tally_tokens(child, root_val)
                    all_vars |= child_vars
                    all_consts |= child_consts
                    all_chars |= child_chars
                    all_strings |= child_strings
            return all_vars, all_consts, all_chars, all_strings
    
    i = 0
    for json_list in json_lists:
        print("new list")
        for prog in json_list:
#             print("I", i)
#             print(prog)
            i += 1
            vars, consts, chars, strings = tally_tokens(prog)
            max_vars = max(max_vars, len(vars))
            max_consts = max(max_consts, len(consts))
            max_chars = max(max_chars, len(chars))
            max_strings = max(max_strings, len(strings))
                
    print(all_tokens)
    print(children)
    print(parents)
    print(max_vars)
    print(max_consts)
    print(max_chars)
    print(max_strings)

In [ ]:
figure_out_csharp(json_lists)

In [ ]:
print("SINGLE PARENTS")
for parent

In [6]:
def get_files(folder, suffix):
    matching_files = []
    for path, dirs, files in os.walk(folder):
        for name in files:
            if name[-len(suffix):] == suffix:
                matching_files.append(name)
    return matching_files

In [ ]:
def haskell_voodoo():
    files = get_files("j2c-raw-data", "cs.txt")
    os.system("mkdir jsonified")
    os.system("ghc CleanCSharpData.hs -main-is CleanCSharpData.main -o CleanCSharpData;")
    for file in files:
        os.system("./CleanCSharpData j2c-raw-data/{} > jsonified/{}".format(file, file))

In [ ]:
# Get all the C# datasets
csharp_files = get_files("jsonified", "cs.txt")

# def blah(file):
#     i = 0
#     with open("jsonified/" + file) as f:
#         lines = f.readlines()
#         for line in lines:
#             data = line.split("\t")
#             if len(data) != 3:
#                 print("weirdness", len(data))
# #                 print(data)
#                 print(i)
#             else:
#                 try:
#                     progs = json.loads(data[2])
# #                     json_lists.append(progs)
#                 except:
#                     pass
#                     print("Invalid json at index", i)
# #                     print(data)
# #                     return
#             i += 1
#     print("done")


# print(csharp_files)





i = 0
total_count = 0
correct_count = 0

json_lists = []
for file in csharp_files:
    j_list = []
    with open("j2c-raw-data/" + file) as f:
        lines = f.readlines()
        for line in lines:
            total_count += 1
            data = line.split("\t")
            if len(data) == 3:
                try:
                    prog = json.loads(data[2])
                    correct_count += 1
                    if not prog == []:
                        j_list.append(prog)
                except:
                    pass
            i += 1
    json_lists.append(j_list)
            
print("TOTAL:", total_count)
print("CORRECT:", correct_count)
print(correct_count/total_count)

In [ ]:
blah("poi-cs.txt")

In [9]:
# Blatantly copied from ProgramMatching.py
def isMatch(jprog, cprog):
    if not (len(jprog) == 3 and len(cprog) == 3):
        return False
    if jprog[2] == "[]" or cprog[2] == "[]":
        return False
    try:
        jName = jprog[1].replace('_','').lower()
        cName = cprog[1].replace('_','').lower()
        jFile = jprog[0][:jprog[0].index('.')].lower()
#         if "/" in jFile:
#             jFile = jFile.split("/")[-1]
        cFile = cprog[0][:cprog[0].index('.')].lower()
#         if "/" in cFile:
#             cFile = cFile.split("/")[-1]
#         print(jName, cName, jFile, cFile)
        return (jName == cName) and (jFile == cFile)
    except:
        print("======JJJJJJJ======")
        print(jprog)
        print("=======CCCCCCC========")
        print(cprog)
        return "weirdness"

def filter_files(jfile, cfile, orig_folder, save_folder):
    d = {}
    
    jlist = []
    clist = []
    with open(orig_folder + "/" + jfile) as f:
        jlist = [line.split("\t") for line in f.readlines()]
        
    with open(orig_folder + "/" + cfile) as f:
        clist = [line.split("\t") for line in f.readlines()]
        
    i = 0
    for jprog in jlist:
        if not i%1000:
            print("i is ", i)
        i+=1
        for cprog in clist:
            match = isMatch(jprog,cprog)
            if match == "weirdness":
                return
            elif match:
                d[jprog[2]] = cprog[2]
    all_files = [(k,v) for k,v in d.items()]
    print("ok", len(all_files))
    # Write java
    with open(save_folder + "/" + jfile, "w") as f:
        for file in all_files:
            f.write(file[0])
    
    # Write C#
    with open(save_folder + "/" + cfile, "w") as f:
        for file in all_files:
            f.write(file[1])
            
        
    

In [11]:
csharp_files = [sorted(get_files("j2c-raw-data", "cs.txt"))[0]]
java_files = [sorted(get_files("j2c-raw-data", "java.txt"))[0]]

print(csharp_files)
print(java_files)

for i in range(len(csharp_files)):
    print("file", java_files[i])
    filter_files(java_files[i], csharp_files[i], "j2c-raw-data", "filtered_data")

jprog = ["/transport/basereceivepack.java", "getAsName", None]
cprog = ['CMapAwareDocumentFont.cs', 'ProcessToUnicode', None]
jName = jprog[1].replace('_','').lower()
cName = cprog[1].replace('_','').lower()
jFile = jprog[0][:jprog[0].index('.')]
cFile = cprog[0][:cprog[0].index('.')]
print("ok")

['antlr-cs.txt']
['antlr-java.txt']
file antlr-java.txt
i is  0
i is  1000
ok 0
ok
